<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

In [3]:
from models.network import Network
import numpy as np

In [4]:
# Let's read the MNIST dataset (handwritten digit recognition)
# 28 x 28 grayscale images: we flatten them here as vectors of length 784 and cast to float64 in 0..1 range.
# More: https://en.wikipedia.org/wiki/MNIST_database

def load_mnist(path: str = 'data/mnist.npz') -> tuple[tuple[np.ndarray, np.ndarray], tuple[np.ndarray, np.ndarray]]:
    with np.load(path) as f:
        x_train, _y_train = f['x_train'], f['y_train']
        x_test, _y_test = f['x_test'], f['y_test']

    x_train = x_train.reshape(-1, 28 * 28) / 255.
    x_test = x_test.reshape(-1, 28 * 28) / 255.

    y_train = np.zeros((_y_train.shape[0], 10))
    y_train[np.arange(_y_train.shape[0]), _y_train] = 1

    y_test = np.zeros((_y_test.shape[0], 10))
    y_test[np.arange(_y_test.shape[0]), _y_test] = 1

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_mnist()
print(x_train.shape, y_train.shape, x_train.dtype, y_train.dtype)
print(x_test.shape, y_test.shape, x_test.dtype, y_test.dtype)

(60000, 784) (60000, 10) float64 float64
(10000, 784) (10000, 10) float64 float64


In [6]:
network = Network([784, 30, 10], loss="log-loss", normalize="softmax")
network.SGD((x_train, y_train), epochs=5, mini_batch_size=100, eta=3., test_data=(x_test, y_test))

Epoch: 0, Accuracy: 0.8864
Epoch: 1, Accuracy: 0.9176
Epoch: 2, Accuracy: 0.9277
Epoch: 3, Accuracy: 0.9326
Epoch: 4, Accuracy: 0.9356


Vectorized version takes <2s per epoch